[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_toolbox/pdftools_toolbox_add_data_matrix.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDKXT/latest/pdftools_toolbox-latest.tar.gz
%pip install ipython

# Add Data Matrix to PDF
Add a two-dimensional barcode from an existing image on
the first page of a PDF document.

In [ ]:
import io
from pdftools_toolbox.pdf import Document, FileReference, Metadata, Page, PageCopyOptions, PageList
from pdftools_toolbox.geometry.real import Rectangle
from pdftools_toolbox.pdf.content import ContentGenerator, IccBasedColorSpace, Image
from pdftools_toolbox.pdf.navigation import ViewerSettings

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/InvoiceNone.pdf'
input_file_path = 'InvoiceNone.pdf'
prepare_file(input_url, input_file_path)
data_matrix_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/DataMatrix.png'
data_matrix_path = 'DataMatrix.png'
prepare_file(data_matrix_url, data_matrix_path)
output_file_path = 'InvoiceWithDataMatrix.pdf'

In [ ]:
def copy_document_data(in_doc: Document, out_doc: Document):
    # Copy document-wide data

    # Output intent
    if in_doc.output_intent is not None:
        in_doc.output_intent = IccBasedColorSpace.copy(out_doc, in_doc.output_intent)

    # Metadata
    out_doc.metadata = Metadata.copy(out_doc, in_doc.metadata)

    # Viewer settings
    out_doc.viewer_settings = ViewerSettings.copy(out_doc, in_doc.viewer_settings)

    # Associated files (for PDF/A-3 and PDF 2.0 only)
    outAssociatedFiles = out_doc.associated_files
    for in_file_ref in in_doc.associated_files:
        outAssociatedFiles.append(FileReference.copy(out_doc, in_file_ref))

    # Plain embedded files
    out_embedded_files = out_doc.plain_embedded_files
    for in_file_ref in in_doc.plain_embedded_files:
        out_embedded_files.append(FileReference.copy(out_doc, in_file_ref))

In [ ]:
def add_data_matrix(document: Document, page: Page, data_matrix_path: str):
    # Create content generator
    with ContentGenerator(page.content, False) as generator:

        #  Import data matrix
        with io.FileIO(data_matrix_path, 'rb') as in_matrix_stream:

            # Create image object for data matrix
            data_matrix = Image.create(document, in_matrix_stream)

            # Data matrix size
            data_matrix_size = 85

            # Calculate Rectangle for data matrix
            rect = Rectangle(left=border, bottom=page.size.height - (data_matrix_size + border), right=data_matrix_size + border, top=page.size.height - border)

        # Paint the positioned barcode text
        generator.paint_image(data_matrix, rect)

In [ ]:
try:
    # Set and check license key. If the license key is not valid, an exception is thrown.
    from pdftools_toolbox.sdk import Sdk
    Sdk.initialize("INSERT-LICENSE-KEY", None)

    # Define border
    border = 40
    
    # Open input document
    with io.FileIO(input_file_path, 'rb') as in_stream:
        with Document.open(in_stream, None) as in_doc:
    
            # Create output document
            with io.FileIO(output_file_path, 'wb+') as output_stream:
                with Document.create(output_stream, in_doc.conformance, None) as out_doc:
    
                    # Copy document-wide data
                    copy_document_data(in_doc, out_doc)
    
                    # Define page copy options
                    copy_options = PageCopyOptions()
    
                    # Copy first page, add datamatrix image, and append to output document
                    out_page = Page.copy(out_doc, in_doc.pages[0], copy_options)
                    add_data_matrix(out_doc, out_page, data_matrix_path)
                    out_doc.pages.append(out_page)
    
                    # Copy remaining pages and append to output document
                    in_page_range = in_doc.pages[1:]
                    copied_pages = PageList.copy(out_doc, in_page_range, copy_options)
                    out_doc.pages.extend(copied_pages)

    print("Execution successful.")
except Exception as e:
    print(f"An error occurred: {e}")